In [19]:
import pandas as pd
import numpy as np
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
import ast
import warnings
import regex as re

warnings.filterwarnings('ignore')
from decimal import *

from enrich_dimensions.rounds import rounds, rounding, re_extract, curate, round_string_float     
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

In [20]:
customer_id = '137'
customer_name='%kimballwest%'
buckets = "Fixings & Fasteners"
# strategy_version_input=705
# attribute_id_input=5131
formatted_attribute = 'tensile_strength_lbs'

In [21]:
# parameters
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [22]:
import json
dfs = query_from_file(file_name='./query/custom_fields.sql', params=params)
custom_field_df=pd.json_normalize(dfs['custom_fields'])
fields = ['TensileStrength_PSI_']
# df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df], axis = 1)
df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df[fields]], axis = 1)
print(len(df))

del df['url']
del df['image_url']
del df['value']
del df['product_id']
del df['bucket_id']

KeyError: "None of [Index(['TensileStrength_PSI_'], dtype='object')] are in the [columns]"

In [5]:
# print(len(df[(df['OutsideDiameter'].astype(str)!='nan')&(df['mm'].astype(str)!='[]')]))
# df[(df['OutsideDiameter'].astype(str)!='nan')&(df['mm'].astype(str)!='[]')]
df.head(0)

Empty DataFrame
Columns: [buckets, external_id, product_name, long_desc, TensileStrength_PSI_]
Index: []

In [6]:
import collections
print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])

[]


In [12]:
# df['MaxPressure'].explode().value_counts()

In [13]:
# start=r'(?i)(^.{0,10})|()'
# df['start']=df['product_name'].apply(lambda x: re_extract(start, str(x)))

pattern=r'''(?i)(\d+\,?\d*.{0,3}lbs?)|()''' 
df['name'] = df['product_name'].apply(lambda x: re_extract(pattern, str(x)))
df['ld'] = df['long_desc'].apply(lambda x: re_extract(pattern, str(x)))



pat=r'(?i)(\d+\,?\d*\s?lbs?)|()'
df['matches']=df['TensileStrength_PSI_'].apply(lambda x: re_extract(pat, str(x)))

# pat=r'(?i)(mm)|()'
# df['mm']=df['OutsideDiameter'].apply(lambda x: re_extract(pat, str(x)))


custom=df[(df['TensileStrength_PSI_'].astype(str)!='nan')]
custom[curation_col]=custom['matches'].apply(lambda x: re.sub(r'.+to\s?|','',str(x))).apply(lambda x: re.sub(r"'|\[|\]",'',str(x)))#.apply(lambda x: re.sub(r'(?i)\s?mm\s?',' mm',str(x))).apply(lambda x: re.sub(r'\s?"\s?',' in',str(x))).apply(lambda x: f'{x} in').apply(lambda x: re.sub(r'\s?in\s?in',' in',str(x)))                                          
print(len(custom))
matchcustom=custom[['external_id',curation_col]]
# custom[curation_col].explode().value_counts()
# # custom[custom[curation_col].astype(str)==r'300 150   ']
custom[0:500]

7


buckets external_id  \
51   Fixings & Fasteners      403952   
52   Fixings & Fasteners      403862   
53   Fixings & Fasteners      403902   
54   Fixings & Fasteners      403802   
55   Fixings & Fasteners      403912   
101  Fixings & Fasteners      356508   
102  Fixings & Fasteners      356516   

                                                          product_name  \
51   KM-USA HH Cap Screw&Torque-Tension Hex Nut Assortment (1/4 - 3/4)   
52   KM-USA HH Cap Screw&Torque-Tension Hex Nut Assortment (1/4 - 3/4)   
53       KM-USA HH Cap Screw&Torque-Tension Nut Assortment (1/4 - 1/2)   
54       KM-USA HH Cap Screw&Torque-Tension Nut Assortment (1/4 - 1/2)   
55       KM-USA HH Cap Screw&Torque-Tension Nut Assortment (1/4 - 5/8)   
101                                  5/16 x 1" Isoplastic Flange Screw   
102                                  5/16 x 2" Isoplastic Flange Screw   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         long_desc  \
51                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      Assortment   
52                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      Assortment   
53                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      Assortment   
54                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      Assortment   
55                               

In [16]:
print(len(df[(df['ld'].astype(str)!='[]')|(df['name'].astype(str)!='[]')]))
print(len(df[(df['ld'].astype(str)=='[]')&(df['name'].astype(str)=='[]')]))
df[(df['ld'].astype(str)!='[]')|(df['name'].astype(str)!='[]')][0:500]

10
2263


buckets external_id  \
849   Fixings & Fasteners       12721   
2200  Fixings & Fasteners      501550   
2203  Fixings & Fasteners      501200   
2211  Fixings & Fasteners      501300   
2213  Fixings & Fasteners      501250   
2220  Fixings & Fasteners      501350   
2225  Fixings & Fasteners      501175   
2230  Fixings & Fasteners      501400   
2233  Fixings & Fasteners      501450   
2242  Fixings & Fasteners      501500   

                                 product_name  \
849                          GM Push Retainer   
2200  #550 Pow'r-Gear® High Torque Hose Clamp   
2203  #200 Pow'r-Gear® High Torque Hose Clamp   
2211  #300 Pow'r-Gear® High Torque Hose Clamp   
2213  #250 Pow'r-Gear® High Torque Hose Clamp   
2220  #350 Pow'r-Gear® High Torque Hose Clamp   
2225  #175 Pow'r-Gear® High Torque Hose Clamp   
2230  #400 Pow'r-Gear® High Torque Hose Clamp   
2233  #450 Pow'r-Gear® High Torque Hose Clamp   
2242  #500 Pow'r-Gear® High Torque Hose Clamp   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   long_desc  \
849                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       G.M. Saturn Parts Rivet-Drive Type Nylon Head 15mm, Diameter 8mm, Length 20mm, Rocker Panel, Wheel Liner, Air Inlet Duct & Front Bumper Fascia 1991 & Up Replaces : 21075686, 11561878, 6504555; Ford F3LB-14K056-AA, F3LY-14570-B   
2200  Designed for high vibration and stubborn leakage applications. Ideal for heavy-duty applications such as industrial machinery, heavy truck, farm and off-road equipment engines. Extra wide 5/8" band provides high strength clamping. Delivers over 800 lbs. of band tension at 150 in. lbs. of screw torque - more than twice the tension of conventional clamps. Wide profile housing provides maximum engagement of screw threads to band. Engineered to deliver exceptionally high rates of band tension to handle even the toughest sealing jobs, including wire reinforced spiral-wound hose. Screw: 5/16" hex-head, 410 stainless steel. Band: 5/8" design, .028 in. thick, 304 stainless steel. Housing: .060 in. thick, 304 stainless steel.   
2203  Designed for high vibration and stubborn leakage applications. Ideal for heavy-duty applications such as industrial machinery, heavy truck, farm and off-road equipment engines. Extra wide 5/8" band provides high strength clamping. Delivers over 800 lbs. of band tension at 150 in. lbs. of screw torque - more than twice the tension of conventional clamps. Wide profile housing provides maximum engagement of screw threads to band. Engineered to deliver exceptionally high rates of band tension to handle even the toughest sealing jobs, including wire reinforced spiral-wound hose. Screw: 5/16" hex-head, 410 stainless steel. Band: 5/8" design, .028 in. thick, 304 stainless steel. Housing: .060 in. thick, 304 stainless steel.   
2211  Designed for high vibration and stubborn leakage applications. Ideal for hea

In [38]:
# df[(df['name'].astype(str)!='[]')&(df['matches'].astype(str)!='[]')][0:500]

In [38]:
two=df[(df['matches'].astype(str)=='[]')]
print(len(two[two['name'].astype(str)!='[]']))
two=two[two['name'].astype(str)!='[]']
two[curation_col]=two['name'].apply(lambda x:re.sub(r'(?i)P\.?S\.?I\.?','',str(x))).apply(lambda x: re.sub(r'.+\-\s?',"['",str(x))).apply(lambda x: re.sub(r'(?<!\,)000',',000',str(x))).apply(lambda x: re.sub(r"\['",'["',str(x))).apply(lambda x: re.sub(r"\s?'\]",'"]',str(x))).apply(lambda x: re.sub(r"\s?\s?\s?'\s?\,\s?'",'","',str(x))).apply(lambda x: re.sub(r'200"\,"125','125","200',str(x))).apply(lambda x: re.sub(r'\["1\,500"\,"100"\,"2,000 "\]','["100","1,500","2,000"]',str(x))).apply(lambda x: re.sub(r'\s?"\]','"]',str(x)))                                                    
two[curation_col].explode().value_counts()
matchtwo=two[['external_id',curation_col]]

1034


In [24]:
# two=df[(df['matches'].astype(str)=='[]')]
# # two['name']=two['name'].apply(lambda x: re.sub(r'(?i)\(|x','',str(x))).apply(lambda x: re.sub(r"'",'"',str(x))).apply(lambda x: re.sub(r'\s?\,\s?',',',str(x))).apply(lambda x: re.sub(r'"\]','""]',str(x))).apply(lambda x:re.sub(r'\[""|\["','["',str(x)))                                                                  
# pat=r'(?i)((?i).{0,20}dia.{0,20})|()'
# two['name']=two['product_name'].apply(lambda x: re_extract(pat, str(x)))
# two['long']=two['long_desc'].apply(lambda x: re_extract(pat, str(x)))
# two[(two['name'].astype(str)!='[]')|(two['long'].astype(str)!='[]')]

# N/A

In [17]:
na=df[(df['name'].astype(str)=='[]')&(df['matches'].astype(str)=='[]')]
print(len(na))
na[curation_col]='n/a'
matchna=na[['external_id',curation_col]]
# print(len(na)+len(matchcustom)+len(matchtwo))

2273


In [34]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchcustom))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchna))
# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchtwo))

# def get_df_name(df):
#     name =[x for x in gglobals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchfinal))

# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchex))

matchcustom
matchna


In [11]:
stop

NameError: name 'stop' is not defined

# send to the folder for upload

In [35]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchcustom) 

In [18]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchna) 

In [47]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchtwo) 